In [1]:
from torchvision import datasets, transforms
import torch
from torch import nn, optim
from sklearn.metrics import accuracy_score
torch.__version__

'1.13.1+cpu'

In [2]:
torch.manual_seed(123)

In [15]:
train = datasets.MNIST(root = '.', train = True, download=True, transform = transforms.ToTensor())
test = datasets.MNIST(root = '.', train = False, download=True, transform = transforms.ToTensor())

In [16]:
train_loader = torch.utils.data.DataLoader(train, batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=256, shuffle=True)

In [5]:
class autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(in_features=784, out_features=32)
        self.activation0 = nn.ReLU()
        self.dense1 = nn.Linear(32, 784)
        self.activation1 = nn.Sigmoid()

    def forward(self, X):
        # Numero inderteminado de registro
        X = X.view(-1, 28*28) # Transdorma a matriz 28*28 em um vetor
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dense1(X)
        X = self.activation1(X)

        return X

In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cpu')

In [7]:
model = autoencoder()
model.to(device)

autoencoder(
  (dense0): Linear(in_features=784, out_features=32, bias=True)
  (activation0): ReLU()
  (dense1): Linear(in_features=32, out_features=784, bias=True)
  (activation1): Sigmoid()
)

In [8]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [9]:
for epoch in range(20):
    running_loss_train = 0.
    for data in train_loader:
        model.train()
        inputs, _ = data 
        inputs = inputs.to(device)
        optimizer.zero_grad()
        outputs = model.forward(inputs)

        loss = criterion(outputs, inputs.view(*outputs.shape)) # transforma os inputs em vetor

        loss.backward()
        optimizer.step()

        running_loss_train += loss.item()

    running_loss_val = 0.
    for data in train_loader:
        model.eval()

        inputs, _ = data
        inputs = inputs.to(device)

        outputs = model.forward(inputs)

        loss = criterion(outputs, inputs.view(*outputs.shape)) # transforma os inputs em vetor

        running_loss_val += loss.item()
    print('ÉPOCA {:3d}: perda_train {:.5f} perda_val {:.5f}'.format(epoch + 1, running_loss_train/len(train_loader), running_loss_val/len(test_loader)))

ÉPOCA   1: perda_train 0.29529 perda_val 1.16992
ÉPOCA   2: perda_train 0.18119 perda_val 0.98222
ÉPOCA   3: perda_train 0.15759 perda_val 0.87673
ÉPOCA   4: perda_train 0.14333 perda_val 0.81137
ÉPOCA   5: perda_train 0.13427 perda_val 0.76727
ÉPOCA   6: perda_train 0.12780 perda_val 0.73547
ÉPOCA   7: perda_train 0.12312 perda_val 0.70835
ÉPOCA   8: perda_train 0.11805 perda_val 0.68231
ÉPOCA   9: perda_train 0.11506 perda_val 0.66903
ÉPOCA  10: perda_train 0.11310 perda_val 0.65957
ÉPOCA  11: perda_train 0.11184 perda_val 0.65377
ÉPOCA  12: perda_train 0.11099 perda_val 0.64956
ÉPOCA  13: perda_train 0.11038 perda_val 0.64730
ÉPOCA  14: perda_train 0.11003 perda_val 0.64480
ÉPOCA  15: perda_train 0.10971 perda_val 0.64060
ÉPOCA  16: perda_train 0.10814 perda_val 0.63228
ÉPOCA  17: perda_train 0.10751 perda_val 0.62971
ÉPOCA  18: perda_train 0.10714 perda_val 0.62776
ÉPOCA  19: perda_train 0.10689 perda_val 0.62707
ÉPOCA  20: perda_train 0.10675 perda_val 0.62644


In [10]:
list(model.children())

[Linear(in_features=784, out_features=32, bias=True),
 ReLU(),
 Linear(in_features=32, out_features=784, bias=True),
 Sigmoid()]

In [11]:
class encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = list(model.children())[0] # Primeira camada
        self.activation0 = list(model.children())[1] # Relu
        
    def forward(self, X):
        X = X.view(-1, 28*28)
        X = self.dense0(X)
        X = self.activation0(X)
        return X

In [12]:
model1 = encoder()
model1.to(device)

encoder(
  (dense0): Linear(in_features=784, out_features=32, bias=True)
  (activation0): ReLU()
)

In [17]:
previsores_treinamento = train.data.float() / 255
previsores_treinamento = previsores_treinamento.view(-1, 784)
previsores_treinamento = previsores_treinamento.to(device)

In [18]:
previsores_treinamento.shape

torch.Size([60000, 784])

In [19]:
with torch.no_grad():
    previsores_treinamento_codificados = model1(previsores_treinamento)
classe_treinamento = train.targets

In [20]:
previsores_treinamento_codificados.shape

torch.Size([60000, 32])

In [21]:
dataset_train_cod = torch.utils.data.TensorDataset(previsores_treinamento_codificados,classe_treinamento)
loader_train_cod = torch.utils.data.DataLoader(dataset_train_cod, batch_size=256, shuffle=True)

In [22]:
previsores_val = test.data.float() / 255
previsores_val = previsores_val.view(-1, 784)
previsores_val = previsores_val.to(device)

In [23]:
with torch.no_grad():
    previsores_val_codificados = model1(previsores_val)
classe_val = test.targets

In [24]:
dataset_test_cod = torch.utils.data.TensorDataset(previsores_val_codificados,classe_val)
loader_val_cod = torch.utils.data.DataLoader(dataset_test_cod, batch_size=256, shuffle=True)

In [25]:
c1 = nn.Sequential(nn.Linear(784, 397),
                   nn.ReLU(),
                   nn.Linear(397, 10),
                   nn.LogSoftmax())
c1.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(c1.parameters())

In [29]:
for epoch in range(20):  
    # Treinamento
    running_loss_train = 0.
    running_accuracy_train = 0.
    for data in train_loader:
        c1.train()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 28*28)
        optimizer.zero_grad()
        outputs = c1(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss_train += loss.item()
        running_accuracy_train += accuracy_score(labels.detach().cpu().numpy(), 
                                                 top_class.detach().cpu().numpy())
    
    # Validação
    running_loss_val = 0.
    running_accuracy_val = 0.
    for data in test_loader:
        c1.eval()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        inputs = inputs.view(-1, 28*28)
        outputs = c1(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        running_loss_val += loss.item()
        running_accuracy_val += accuracy_score(labels.detach().cpu().numpy(), 
                                               top_class.detach().cpu().numpy())
        
    # Final da época
    print(f'ÉPOCA {epoch + 1}: perda_train {running_loss_train/len(train_loader)} precisão_train {running_accuracy_train/len(train_loader)} perda_val {running_loss_val/len(test_loader)} precisão_val {running_accuracy_val/len(test_loader)}')

c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

ÉPOCA 1: perda_train 0.08650011921975206 precisão_train 0.9750941932624113 perda_val 0.09288303041830659 precisão_val 0.96953125


c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

ÉPOCA 2: perda_train 0.06732554703633836 precisão_train 0.9806903812056738 perda_val 0.08351608971133828 precisão_val 0.97333984375


c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

...

c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

ÉPOCA 19: perda_train 0.002243502655789811 precisão_train 0.9999168882978723 perda_val 0.06861920405644924 precisão_val 0.98203125


c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

ÉPOCA 20: perda_train 0.0017466892023877973 precisão_train 0.9999833776595745 perda_val 0.07160040482413024 precisão_val 0.980078125


c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

In [30]:
c2 = nn.Sequential(
        nn.Linear(32, 21),
        nn.ReLU(),
        nn.Linear(21, 10),
        nn.LogSoftmax())
c2.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(c2.parameters())

In [31]:
for epoch in range(20): 
    running_loss_train = 0.
    running_accuracy_train = 0.
    for data in loader_train_cod:
        c2.train()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = c2(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss_train += loss.item()
        running_accuracy_train += accuracy_score(labels.detach().cpu().numpy(), 
                                                 top_class.detach().cpu().numpy())
    
    # Validação
    running_loss_val = 0.
    running_accuracy_val = 0.
    for data in loader_val_cod:
        c2.eval()
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = c2(inputs)
        ps = torch.exp(outputs)
        _, top_class = ps.topk(k = 1, dim = 1)
        loss = criterion(outputs, labels)
        running_loss_val += loss.item()
        running_accuracy_val += accuracy_score(labels.detach().cpu().numpy(), 
                                               top_class.detach().cpu().numpy())
        
    # Final da época
    print(f'ÉPOCA {epoch + 1}: perda_train {running_loss_train/len(train_loader)} precisão_train {running_accuracy_train/len(train_loader)} perda_val {running_loss_val/len(test_loader)} precisão_val {running_accuracy_val/len(test_loader)}')

c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

ÉPOCA 1: perda_train 1.4868384145675821 precisão_train 0.5270223847517731 perda_val 0.6963036298751831 precisão_val 0.808203125


c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

ÉPOCA 2: perda_train 0.5737477757829301 precisão_train 0.8342918882978724 perda_val 0.46231675073504447 precisão_val 0.86552734375


c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

...

c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

ÉPOCA 19: perda_train 0.2701580248614575 precisão_train 0.919425975177305 perda_val 0.2544028803706169 precisão_val 0.92177734375


c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume

ÉPOCA 20: perda_train 0.2650393330036326 precisão_train 0.9205341312056738 perda_val 0.25742947645485403 precisão_val 0.92333984375


c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
c:\Users\vitao\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\container.py:204: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argume